<h1 style="text-align: center;">Project No. 1 - N-Grams & Sentiment Analysis</h1>

<h4 style="text-align: center;">

Esteban Gomez Valerio

Roi Jared Flores Garza Stone

Rafael Takata Garcia

Text Mining - O2025_MAF3654H

Ing. Juan Antonio Vega Fernández, M. Sc., M. T. Ed

ITESO
</h4>

#### Libraries

In [ ]:
import pandas as pd
from datasets import load_dataset

#### Dataset

In [ ]:
imdb_data = load_dataset("imdb")

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [38]:
train = pd.DataFrame(imdb_data['train'])
test = pd.DataFrame(imdb_data['test'])
dev_data = pd.DataFrame(imdb_data['unsupervised'])
train.shape, test.shape, dev_data.shape

((25000, 2), (25000, 2), (50000, 2))